In [17]:
# Utilizing the Google Maps and Places API to answer questions

import requests
import json

from config import key

geo_url = 'https://maps.googleapis.com/maps/api/geocode/json?address='
place_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='


In [2]:
# Geocoordinates (latitude/longitude) of Seattle, Washington

url = geo_url + 'Seattle,+Washington&key=%s' % (key)
response = requests.get(url).json()
latitude = response['results'][0]['geometry']['location']['lat']
longitude = response['results'][0]['geometry']['location']['lng']
print(f'The GeoCoordinates for Seattle, Washington are {latitude},{longitude}.')

The GeoCoordinates for Seattle, Washington are 47.6062095,-122.3320708.


In [3]:
# What are the geocoordinates (latitude/longitude) of The White House?

url = geo_url + 'White,+House&key=%s' % (key)
response = requests.get(url).json()

lat = response['results'][0]['geometry']['location']['lat']
long = response['results'][0]['geometry']['location']['lng']

print(f'The GeoCoordinates for The White House are {lat},{long}.')

The GeoCoordinates for The White House are 38.8976763,-77.0365298.


In [4]:
# Find the name and address of a bike store in Seattle, Washington.

url = place_url + f'{latitude}, {longitude}&radius=1500&type=bicycle_store&key=%s' % (key)
response = requests.get(url).json()
name = response['results'][0]['name']
address = response['results'][0]['vicinity']
open_now = response['results'][0]['opening_hours']['open_now']

print(f'The name of the bike store is {name} and their address is {address}.  They are open now: {open_now}.')

The name of the bike store is The Bicycle Repair Shop and their address is 68 Madison Street, Seattle.  They are open now: True.


In [5]:
# Find a balloon store near the White House.
url = place_url + f'{lat}, {long}&radius=1500&type=florist&key=%s' % (key)
response = requests.get(url).json()
name = response['results'][0]['name']
address = response['results'][0]['vicinity']
open_now = response['results'][0]['opening_hours']['open_now']
print(f'The name of the balloon store is {name} and their address is {address}.  They are open now: {open_now}.')

The name of the balloon store is GreenWorks Flowers and their address is 1455 Pennsylvania Avenue Northwest, Washington.  They are open now: True.


In [6]:
# Find the nearest dentist to your house.

url = geo_url + '5035+Andrea+st,+Houston,+TX&key=%s' % (key)

response = requests.get(url).json()

lat = response['results'][0]['geometry']['bounds']['northeast']['lat']
long = response['results'][0]['geometry']['bounds']['northeast']['lng']

url = place_url +f'{lat}, {long}&rankby=distance&type=dentist&key=%s' % (key)

response = requests.get(url).json()
name = response['results'][0]['name']

print(f'The nearest Dentist to my House is {name}.')

The nearest Dentist to my House is Harris Isaiah L DDS.


In [7]:
# Find the rating of every airport in the top 100 Metropolitan areas

import pandas as pd
url = 'https://maps.googleapis.com/maps/api/geocode/json'
place_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Read in a CSV of City and State Data
cities = pd.read_csv('Cities.csv')
print(cities.head())

# Find the Latitude and Longitude for each city using the Google GeoCode API
for index, row in cities.iterrows():
    city = row['City']
    state = row['State']
    
    params = {
        'address':f'{city}, {state}',
        'key':key}
    
    response = requests.get(url,params=params).json()
    
    lat = response['results'][0]['geometry']['location']
    long = response['results'][0]['geometry']['location']
    
    cities.loc[index, 'Lat'] = lat.get('lat', '')
    cities.loc[index,'long'] = long.get('lng','')
    
    lat = lat.get('lat','')
    long = long.get('lng','')
    
    # Use the Google Places API to find the rating, name, and address of the nearest airport to the Coordinates
    params = {
        'location': f'{lat},{long}',
        'key':key,
        'type':'airport',
        'radius':50000,
        'keyword':'international airport'
    }
    
    response = requests.get(place_url,params=params).json()
    result = response.get('results')
    
    try:
        nearest_airport = result[0]
        cities.loc[index,'Name'] = response['results'][0]['name']
        cities.loc[index,'Address'] = response['results'][0]['vicinity']
        cities.loc[index,'rating'] = response['results'][0]['rating']
    except:
        print(f'No Results for {city}')
    

In [8]:
# Utilize Gmaps to plot geographical data in the form of a heatmap

import gmaps

locations = cities[['Lat','long']]
rating = cities['rating'].astype(float)
locations = locations.dropna()
rating = rating.dropna()

gmaps.configure(api_key=key)
fig = gmaps.figure()

fig.add_layer(gmaps.heatmap_layer(locations, weights=rating, dissipating=False,max_intensity=10,point_radius=1))

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Utilize the Census API to gather data for the year 2013

from census import Census

api_key = "85ac64b6b5a9c0901b00329d1ef41f0c53ccfc98"

c = Census(api_key,year=2013)

# Send a Call for State information regarding Household Income, Population, Median Age, Per Capita Income, and Poverty Count
data = c.acs5.get(('NAME','B19013_001E','B01003_001E','B01002_001E','B19301_001E','B17001_002E'), {'for':'state:*'})

data_df = pd.DataFrame(data)

data_df = data_df.rename(columns={'NAME':'Name','state':'State',
                                'B19013_001E':'Household Income',
                                 'B01003_001E':'Population',
                                 'B01002_001E':'Median Age',
                                 'B19301_001E':'Per Capita Income',
                                 'B17001_002E':'Poverty Count'})
# Create the Column Poverty Rate from the Poverty Count and Population columns
data_df['Poverty Rate'] = data_df['Poverty Count'].astype(int)/data_df['Population'].astype(int) * 100

data_df.set_index('State')

data_df = data_df[["Name","Population",'Median Age','Poverty Count','Per Capita Income','Household Income','Poverty Rate']]

In [18]:
# Reading in a File that Contains the Lat, Long for each state and merging it with the previous dataframe on the State Name
state_locations = pd.read_csv('state_centroids.csv')
state_loc = state_locations.rename(columns={'State':'Name'})

data_d = data_df.merge(state_loc,on='Name',how='inner')

# Plotting the States
data_d['Latitude'] = data_d['Latitude'].astype(float)
data_d['Longitude'] = data_d['Longitude'].astype(float)

p_rate = data_d['Poverty Rate']

location = data_d[['Latitude','Longitude']]

fig = gmaps.figure()

# Plotting the Lat and Long for each state and using the Poverty Rate as the weight for the Heatmap
fig.add_layer(gmaps.heatmap_layer(location, weights=data_d['Poverty Rate'],dissipating=False,max_intensity=25,point_radius=3))

fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Using all the above technologies to showcase the correlation between low-income areas and the absence of banking services

import pandas as pd
import requests 
import json
from config import key
import gmaps
from census import Census

census_key = "85ac64b6b5a9c0901b00329d1ef41f0c53ccfc98"
c = Census(census_key,year=2013)

# Read in Data that contains the number of banking services by zipcode
data = pd.read_csv('zip_bank_data.csv')

# Use the Census API to obtain Unemployment numbers by zipcode
census_pd = c.acs5.get(('B23025_005E'),
                       {'for':'zip code tabulation area'})
census_pd = pd.DataFrame(census_pd)
census_pd = census_pd.rename(columns={'B23025_005E':'Unemployment',
                                     'zip code tabulation area':'Zipcode'})
census_pd.head()

,Unemployment,Zipcode
0,1464.0,00601
1,4767.0,00602
2,4341.0,00603
3,134.0,00606
4,1134.0,00610


In [20]:
# Merge the two datasets on the zipcode 
data['Zipcode'] = pd.to_numeric(data['Zipcode'])
census_pd['Zipcode'] = pd.to_numeric(census_pd['Zipcode'])

data_full = data.merge(census_pd,on='Zipcode',how='inner')

# Create the Unemployment Rate by using both the Unemployment and Population column 
data_full['Unemployment Rate'] = 100 * data_full['Unemployment']/data_full['Population']
data_full.head()

,Zipcode,Address,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Lat,Lng,Bank Count,Unemployment,Unemployment Rate
0,49058,"Hastings, MI 49058, USA",19031,40.8,46777.0,22137,13.183753,42.630692,-85.292938,9,1011.0,5.312385
1,71405,"Ball, LA 71405, USA",5687,35.8,55242.0,23941,15.737647,31.406180,-92.396174,12,260.0,4.571830
2,20634,"Great Mills, MD 20634, USA",6562,31.9,79944.0,35961,9.295946,38.220161,-76.496792,9,266.0,4.053642
3,38487,"Williamsport, TN 38487, USA",762,41.6,38125.0,18884,24.409449,35.731037,-87.241930,0,78.0,10.236220
4,58466,"Marion, ND 58466, USA",509,44.5,69844.0,36981,9.430255,46.559422,-98.348154,1,6.0,1.178782


In [23]:
# Plot using a heatmap the Unemployment rate for each zipcode
fig = gmaps.figure()
location = data_full[['Lat','Lng']]
markers = gmaps.marker_layer(location)
fig.add_layer(gmaps.heatmap_layer(location, weights=data_full['Unemployment Rate'],
                                 max_intensity = 50,
                                  dissipating=False,point_radius=1))

fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Use a Symbol Layer to showcase the amount of banks are in a specific zipcode
bank_rate = data_full['Bank Count'].tolist()

fig = gmaps.figure()
layer = gmaps.symbol_layer(location,
                           fill_color='rgba(0,150,0,0.4)',
                           stroke_color='rgba(0,0,150,0.4)',scale=2,
                          info_box_content=[f'# of Banks In Area: {bank}'for bank in bank_rate])

fig.add_layer(layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# Combine Both Maps from Above into One
fig = gmaps.figure()

fig.add_layer(gmaps.heatmap_layer(location, weights=data_full['Unemployment Rate'],
                                 max_intensity = 50,
                                  dissipating=False,point_radius=1))
fig.add_layer(layer)

fig

Figure(layout=FigureLayout(height='420px'))